In [78]:
import pandas as pd
import sklearn
import os
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
%matplotlib inline

In [79]:
data=pd.read_csv('data_FeatENG.csv')
data.drop('Unnamed: 0',axis=1,inplace=True)
print(data.head())
data.describe()
X=data.drop('Allotment',axis=1)
y=data['Allotment']

   RANK SEAT CATEGORY Allotment
0   144     OC_GEN_UR  MVSR_CSE
1   233     OC_GEN_UR  MVSR_CSE
2   243   BC_B_GEN_UR  MVSR_CSE
3   286    EWS_GEN_UR  MVSR_CSE
4   318   BC_D_GEN_OU  JNTH_CSE


In [80]:
clg=[]
for ind,val in enumerate(data["Allotment"].value_counts()==1):
    if val:
        clg.append((data["Allotment"].value_counts()==1).keys()[ind])
        print((data["Allotment"].value_counts()==1).keys()[ind])


data.drop(data.loc[data['Allotment'].isin(clg)].index,axis=0,inplace=True)

seat=[]
for ind,val in enumerate(data['SEAT CATEGORY'].value_counts()==1):
    if val:
        seat.append((data['SEAT CATEGORY'].value_counts()==1).keys()[ind])
        print((data['SEAT CATEGORY'].value_counts()==1).keys()[ind])


data.drop(data.loc[data['SEAT CATEGORY'].isin(seat)].index,axis=0,inplace=True)

print(len( data['SEAT CATEGORY'].value_counts()))
X=data.drop('Allotment',axis=1)
y=data['Allotment']


x_train,x_test,y_train,y_test=train_test_split(X,y,stratify=X['SEAT CATEGORY'])

print(len(x_train))

VCET_EEE
VCET_MEC
BIET_MEC
RITW_ECE
MGHA_EEE
GURU_PHM
VVKN_EEE
GLOB_MEC
HOLY_CSW
CVSR_PHD
SRYS_MEC
MOTK_MIN
TCTK_EEE
NREC_MEC
KMTS_CIV
SVES_MEC
NREC_CIV
MTEC_MEC
MREM_MEC
NNRG_PHM
SC_SG_GEN_OU(151)
BC_C_PHO_GIRLS_OU
BC_B_SG_GEN_OU(175)
BC_B_NCC_GIRLS_OU(57)
BC_C_PHH_GEN_OU
BC_E_SG_GEN_OU(127)
ST_NCC_GIRLS_OU(56)
SC_SG_GEN_OU(172)
BC_C_SG_GIRLS_OU(0)
BC_E_SG_GEN_OU(128)
OC_SG_GIRLS_OU(179)
OC_NCC_GEN_OU(29)
OC_SG_GEN_OU(176)
OC_NCC_GEN_OU(20)
BC_B_NCC_GEN_OU(56)
OC_SG_GEN_OU(151)
OC_NCC_GEN_OU(33)
BC_A_SG_GIRLS_OU(128)
OC_NCC_GEN_OU(21)
OC_SG_GIRLS_OU(175)
OC_SG_GIRLS_OU(143)
BC_A_NCC_GEN_OU(56)
OC_CAP_GIRLS_UR(3)
OC_NCC_GEN_OU(41)
SC_SG_GEN_OU(128)
OC_SG_GEN_OU(148)
BC_A_SG_GEN_OU(174)
SC_SG_GIRLS_OU(127)
OC_SG_GIRLS_OU(112)
BC_A_SG_GIRLS_OU(0)
166
37030


In [81]:
enc=OneHotEncoder(handle_unknown='ignore')


encoder = ColumnTransformer([
    ('cat_encoder', enc, ['SEAT CATEGORY'])
],)


pipe = Pipeline([
    ('encoder', encoder),
    ("clf",KNeighborsClassifier())])
pipe['clf'].get_params().keys()

dict_keys(['algorithm', 'leaf_size', 'metric', 'metric_params', 'n_jobs', 'n_neighbors', 'p', 'weights'])

In [82]:
from skopt import BayesSearchCV
from skopt.space import Real,Integer

search_space = {
    'clf__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'clf__leaf_size': Integer(1,10),
    'clf__n_neighbors': Integer(1,120),
    'clf__metric': ['cityblock', 'cosine', 'euclidean', 'l1', 'l2', 'manhattan', 'correlation'],
    'clf__p': Real(0, 3),
    'clf__weights': ['uniform','distance'],}

opt = BayesSearchCV(pipe, search_space, cv=2, n_iter=1, scoring='accuracy') 

In [83]:
opt.fit(X,y)

c:\Users\user\anaconda3\lib\site-packages\sklearn\neighbors\_base.py:504: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")
c:\Users\user\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
c:\Users\user\anaconda3\lib\site-packages\sklearn\neighbors\_base.py:504: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")
c:\Users\user\anaconda3\lib\sit

BayesSearchCV(cv=2,
              estimator=Pipeline(steps=[('encoder',
                                         ColumnTransformer(transformers=[('cat_encoder',
                                                                          OneHotEncoder(handle_unknown='ignore'),
                                                                          ['SEAT '
                                                                           'CATEGORY'])])),
                                        ('clf', KNeighborsClassifier())]),
              n_iter=1, scoring='accuracy',
              search_spaces={'clf__algorithm': ['auto', 'ball_tree', 'kd_tree',
                                                'brute'],
                             'clf__leaf_size': Integer(low=1, high=10, prior='uniform', transform='normalize'),
                             'clf__metric': ['cityblock', 'cosine', 'euclidean',
                                             'l1', 'l2', 'manhattan',
                              

In [84]:
best=opt.best_estimator_

In [85]:
opt.best_params_

OrderedDict([('clf__algorithm', 'ball_tree'),
             ('clf__leaf_size', 10),
             ('clf__metric', 'l2'),
             ('clf__n_neighbors', 13),
             ('clf__p', 1.293124401202159),
             ('clf__weights', 'uniform')])

In [93]:
enc.fit(X)
inp=pd.DataFrame({'SEAT CATEGORY':['OC_GEN_OU']})
inp=enc.transform(inp)
inp=pd.DataFrame(inp)
inpn=pd.DataFrame({'RANK':[10101]})
inpn.index=inp.index
inp=pd.concat([inpn,inp],axis=1)

c:\Users\user\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names seen at fit time, yet now missing:
- RANK

  warnings.warn(message, FutureWarning)


ValueError: X has 1 features, but OneHotEncoder is expecting 2 features as input.

In [94]:
best.predict_proba(inp)

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.07692308, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.07692308, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.07692308, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

False